# 1.Colab 환경 설정


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp38-cp38-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.8.1
    Uninstalling transformers-4.8.1:
      Successfully uninstalled transformers-4.8.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-vu4x4p69
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-vu4x4p69
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Using cached transformers-4.8.1-py3-none-any.whl (2.5 MB)
  Using cached tokenizers-0.10.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


# 2.데이터셋 전처리

In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Train

In [ ]:
import pandas as pd
chatbot_data = pd.read_csv('/content/drive/MyDrive/kobert/9keywords_train.csv')

In [ ]:
chatbot_data.sample(n=10)

,상황키워드,감정_대분류,사람문장
5065,학교폭력/따돌림,분노,요즘 학교에서 따돌림을 당하고 있어. 너무 힘들어. 아니. 말하고 싶지 않아. 그냥...
37616,학교폭력/따돌림,기쁨,우리 학교는 학교폭력과 따돌림이 없어서 정말 흡족해. 학교에 학교폭력과 따돌림이 존...
2326,대인관계,당황,회의 중에 부장님과 의견 충돌이 있었는데 내가 너무 언성을 높인 것 같아 죄책감이 ...
34974,"진로,취업,직장",기쁨,이번 계약을 성공적으로 성사시키면서 회사 사람들로부터 능력을 인정받아서 너무 기뻐....
37837,학교폭력/따돌림,상처,아직 같은 반에 친구가 없어. 부모님께 전학 보내달라고 요청할까? 차라리 이유를 알...
18260,"진로,취업,직장",기쁨,팀원들과 열심히 준비한 이 프로젝트는 성공할 것이야. 팀원들이 잘 따라와 줬어. 열...
2505,대인관계,슬픔,친구들이 내 뒤에서 내 뒷말을 하는 걸 들었어. 십년지기 친구들인데. 내 대인관계에...
4143,"진로,취업,직장",불안,면접이 얼마 남지 않아 걱정이야. 면접관 앞에서 너무 떨려 말을 잘하지 못할까 봐 ...
48860,가족관계,불안,자식이랑 부모님 모두 책임지는 게 너무 힘들어. 내 장애 때문에 나이 들수록 일하는...
26474,"연애,결혼,출산",상처,결혼을 하려던 사람과 헤어졌어. 너무나 큰 상처를 받았어. 새로운 사람을 만나 다시...


In [ ]:
chatbot_data = chatbot_data.rename(columns={'상황키워드': 'Situation', '사람문장': 'Sentence'})

In [ ]:
chatbot_data

,Situation,감정_대분류,Sentence
0,"진로,취업,직장",분노,일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...
1,"진로,취업,직장",분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....
2,"진로,취업,직장",분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,"진로,취업,직장",분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,"진로,취업,직장",분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...
...,...,...,...
51625,"재정,은퇴,노후준비",분노,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고....
51626,"재정,은퇴,노후준비",불안,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나. 마음...
51627,"재정,은퇴,노후준비",상처,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정...
51628,대인관계,불안,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가...


In [ ]:
chatbot_data['Situation'].unique()

array(['진로,취업,직장', '대인관계', '연애,결혼,출산', '재정,은퇴,노후준비', '건강,죽음', '학업 및 진로',
       '학교폭력/따돌림', '직장, 업무 스트레스', '가족관계'], dtype=object)

In [ ]:
chatbot_data.loc[(chatbot_data['Situation'] == "대인관계"), 'Situation'] = 0  #대인관계 => 0
chatbot_data.loc[(chatbot_data['Situation'] == "진로,취업,직장"), 'Situation'] = 1  #진로,취업,직장 => 1
chatbot_data.loc[(chatbot_data['Situation'] == "연애,결혼,출산"), 'Situation'] = 2  #연애,결혼,출산 => 2
chatbot_data.loc[(chatbot_data['Situation'] == "가족관계"), 'Situation'] = 3  #가족관계 => 3
chatbot_data.loc[(chatbot_data['Situation'] == "학업 및 진로"), 'Situation'] = 4  #학업 및 진로 => 4
chatbot_data.loc[(chatbot_data['Situation'] == "학교폭력/따돌림"), 'Situation'] = 5  #학교폭력/따돌림 => 5
chatbot_data.loc[(chatbot_data['Situation'] == "재정,은퇴,노후준비"), 'Situation'] = 6  #재정,은퇴,노후준비 => 6
chatbot_data.loc[(chatbot_data['Situation'] == "직장, 업무 스트레스"), 'Situation'] = 7  #직장, 업무 스트레스 => 7
chatbot_data.loc[(chatbot_data['Situation'] == "건강,죽음"), 'Situation'] = 8  #건강,죽음 => 8

data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Situation'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고. .', '1']
['최근에 업무가 너무 많이 늘어난 것 같아 힘들어. 안 그래도 할 일 많은데 왜 자꾸 나한테 추가 업무를 주는지 모르겠다니깐. .', '7']
['친구들은 다 취업에 성공했는데 나만 못한 것 같아. 나도 졸업하고 바로 일하고 싶었는데. 응. 나는 다 잘 될 줄 알았는데 이게 뭐야.', '1']
['친구에게 내가 간암에 걸려서 술을 마실 수 없다고 하자 거짓말하지 말라고 했어. 간암에 걸렸다는 말을 왜 안 믿을까? 친구가 믿지 않는 건 어쩔 수 없지만 조금 서운하네.', '0']
['의사가 분명히 수술이 잘 되었다고 했거든. 그런데 삼 주가 되도록 몸을 움직일 수 없어. 그러니 불안해서 견딜 수가 있어야지. 아예 움직일 수 없을까 봐. 아직 포기하긴 이르지. 할 수 있는 모든 방법을 다 써봐야지.', '8']
['이번에 주식을 샀는데 주가가 너무 떨어져 손해가 이만저만이 아냐. 지금은 없어. 난 왜 이렇게 되는일이 없나 몰라. .', '8']
['남편과 결혼한 지 사십 년이야. 이제 사람 만나는 것도 버겁고 알던 사람도 점점 사라져. 사람들을 만나는 것이 어려워. 자꾸 사람들을 의심하게만 되고 말이야. 사람들을 볼 때 의심하고 불신하는 마음을 억눌러야겠어. 사람들을 색안경을 끼고 보지 않게.', '0']


In [ ]:
dataset_train = data_list

# test

In [ ]:
import pandas as pd
chatbot_data = pd.read_csv('/content/drive/MyDrive/kobert/9keywords_validation.csv')

In [ ]:
chatbot_data = chatbot_data.rename(columns={'상황키워드': 'Situation', '사람문장': 'Sentence'})

In [ ]:
chatbot_data.loc[(chatbot_data['Situation'] == "대인관계"), 'Situation'] = 0  #대인관계 => 0
chatbot_data.loc[(chatbot_data['Situation'] == "진로,취업,직장"), 'Situation'] = 1  #진로,취업,직장 => 1
chatbot_data.loc[(chatbot_data['Situation'] == "연애,결혼,출산"), 'Situation'] = 2  #연애,결혼,출산 => 2
chatbot_data.loc[(chatbot_data['Situation'] == "가족관계"), 'Situation'] = 3  #가족관계 => 3
chatbot_data.loc[(chatbot_data['Situation'] == "학업 및 진로"), 'Situation'] = 4  #학업 및 진로 => 4
chatbot_data.loc[(chatbot_data['Situation'] == "학교폭력/따돌림"), 'Situation'] = 5  #학교폭력/따돌림 => 5
chatbot_data.loc[(chatbot_data['Situation'] == "재정,은퇴,노후준비"), 'Situation'] = 6  #재정,은퇴,노후준비 => 6
chatbot_data.loc[(chatbot_data['Situation'] == "직장, 업무 스트레스"), 'Situation'] = 7  #직장, 업무 스트레스 => 7
chatbot_data.loc[(chatbot_data['Situation'] == "건강,죽음"), 'Situation'] = 8  #건강,죽음 => 8

data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Situation'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
dataset_test = data_list

# 3.Train data & Test data


In [ ]:
print(len(dataset_train))
print(len(dataset_test))

51630
6641


# 4.KoBERT 입력 데이터로 만들기

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# 5.KoBERT 학습모델 만들기


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=9,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

# 6.KoBERT 모델 학습시키기


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-37-480b6a139979>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/807 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.4037630558013916 train acc 0.046875
epoch 1 batch id 201 loss 2.2261109352111816 train acc 0.3670708955223881
epoch 1 batch id 401 loss 1.0957101583480835 train acc 0.47089307980049877
epoch 1 batch id 601 loss 1.5753934383392334 train acc 0.5360076955074875
epoch 1 batch id 801 loss 0.7241837382316589 train acc 0.5624024656679151
epoch 1 train acc 0.5643907251764453


<ipython-input-37-480b6a139979>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/104 [00:00<?, ?it/s]

...

epoch 4 test acc 0.634630715266876


  0%|          | 0/807 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.806276798248291 train acc 0.6875
epoch 5 batch id 201 loss 1.1524728536605835 train acc 0.8260261194029851
epoch 5 batch id 401 loss 0.8667235374450684 train acc 0.7946929551122195
epoch 5 batch id 601 loss 0.9114647507667542 train acc 0.8044405158069884
epoch 5 batch id 801 loss 1.1160866022109985 train acc 0.7745396379525593
epoch 5 train acc 0.7738520957922527


  0%|          | 0/104 [00:00<?, ?it/s]

epoch 5 test acc 0.7768378385007849


# 7.새로운 문장 테스트


In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("대인관계")
            elif np.argmax(logits) == 1:
                test_eval.append("진로, 취업, 직장")
            elif np.argmax(logits) == 2:
                test_eval.append("연애, 결혼, 출산")
            elif np.argmax(logits) == 3:
                test_eval.append("가족관계")
            elif np.argmax(logits) == 4:
                test_eval.append("학업 및 진로")
            elif np.argmax(logits) == 5:
                test_eval.append("학교폭력 따돌림")
            elif np.argmax(logits) == 6:
                test_eval.append("재정, 은퇴,노후준비")
            elif np.argmax(logits) == 7:
                test_eval.append("직장, 업무스트레스")
            elif np.argmax(logits) == 8:
                test_eval.append("건강죽음")

        print(">> 입력하신 내용은 " + test_eval[0] + " 상황입니다.")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 민호형 취뽀
>> 입력하신 내용은 대인관계 상황입니다.


하고싶은 말을 입력해주세요 : 민호형 취업했어
>> 입력하신 내용은 진로, 취업, 직장 상황입니다.


하고싶은 말을 입력해주세요 : 엄마 생일이ㅑ
>> 입력하신 내용은 가족관계 상황입니다.


하고싶은 말을 입력해주세요 : 시험 망했어
>> 입력하신 내용은 학업 및 진로 상황입니다.


하고싶은 말을 입력해주세요 : 할아버지 기일이야
>> 입력하신 내용은 가족관계 상황입니다.


하고싶은 말을 입력해주세요 : 나 암 말기래
>> 입력하신 내용은 건강죽음 상황입니다.


하고싶은 말을 입력해주세요 : 아버지가 곧 정년퇴직
>> 입력하신 내용은 재정, 은퇴,노후준비 상황입니다.


하고싶은 말을 입력해주세요 : 0


In [ ]:
import torch
import h5py

# 모델을 저장할 경로를 지정합니다.
PATH = "/content/drive/MyDrive/kobert/kobert_situation_epoch5.h5"

# 모델의 state_dict를 저장합니다.
torch.save(model.state_dict(), PATH)

# h5py를 사용하여 .h5 파일로 저장합니다.
with h5py.File(PATH, "w") as f:
    for key in model.state_dict():
        f.create_dataset(key, data=model.state_dict()[key].cpu().numpy())

In [ ]:
import torch
import h5py
import numpy as np

# 모델 아키텍처를 정의합니다.
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# 모델을 로드할 경로를 지정합니다.
PATH = "/content/drive/MyDrive/kobert/kobert_situation_epoch5.h5"

# .h5 파일을 열어서 state_dict를 로드합니다.
with h5py.File(PATH, "r") as f:
    for key in f.keys():
        param = torch.from_numpy(np.array(f[key]))
        model.state_dict()[key].copy_(param)


In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")